# Diagnostic Analysis

Analysis of orbital debris dataset. Generates tables to verify key findings about object categories, operators, and zombie satellites. AI helped me decide on types of reports I should include.

## Setup: Load Master Registry

In [1]:
# This is all of our imports and data loading.

import pandas as pd
import numpy as np
from datetime import datetime

data_path = '../data/clean/kinetic_master.csv'
master = pd.read_csv(data_path, low_memory=False)

print(f"Loaded kinetic_master.csv: {len(master):,} records")

Loaded kinetic_master.csv: 32,838 records


## 1. Data Quality Audit

In [2]:
# Data Quality Summary
# This report provides an overview of the data quality for each column in the dataset.
# This allows me to quickly spot missing data, wrong data types, etc.
# every row represents a column in the dataset, I don't need ai for this.
# All were doing is binding together some basic pandas functions to get a summary of each column.
quality_audit = pd.DataFrame({
    'Column': master.columns,
    'Data Type': master.dtypes.values,
    'Non-Null Count': master.count().values,
    'Null Count': master.isnull().sum().values,
    'Null %': (master.isnull().sum().values / len(master) * 100).round(2), # 2 decimal places
    'Unique Values': [master[col].nunique() for col in master.columns]
})

print("\n" + "="*100)
print("DATA QUALITY AUDIT")
print("="*100)
print(quality_audit.to_string(index=True))
print(f"\n✓ Total Records: {len(master):,}") # length of the dataframe is the number of records
print(f"✓ Total Features: {len(master.columns)}") # features are just a 'nice' way of saying columns
print(f"✓ Average Data Completeness: {(master.count().sum() / (len(master) * len(master.columns)) * 100):.2f}%")


DATA QUALITY AUDIT
                 Column Data Type  Non-Null Count  Null Count  Null %  Unique Values
0              norad_id     int64           32838           0    0.00          32838
1             cospar_id    object           32838           0    0.00          32838
2           object_name    object           32838           0    0.00          18253
3        satellite_name    object            5588       27250   82.98           5575
4         official_name    object            5588       27250   82.98           5566
5              category    object           32838           0    0.00              5
6           object_type    object           32838           0    0.00              4
7        launch_mass_kg   float64            5588       27250   82.98            562
8         proxy_mass_kg   float64           32838           0    0.00            563
9           dry_mass_kg   float64           32838           0    0.00            611
10          power_watts   float64            

## 2. Category Breakdown Report

In [3]:
# Category Analysis
in_orbit = master[master['in_orbit'] == 1] # create a new dataframe using in_orbit as a filter

# Use group by to separate by category(active sats, debris, etc). Basically the categories become the rows,
# and then we specify the columns we want to aggregate and the functions to apply to each column.
# column: [agg funcs]
category_breakdown = in_orbit.groupby('category').agg({
  # this order matters, it will be the order of the columns in the final output
    'norad_id': 'count',  # 0
    'proxy_mass_kg': ['sum', 'mean', 'median'], # 1 , 2, 3
    'kinetic_joules': ['sum', 'mean'], # 4, 5
    'launch_year': 'mean' # 6

    # df['column'].agg(['sum', 'mean', 'median']) is another way to do it for a single column
    #
    # df[0]
}).round(2)

# rename the columns, group by creates multi index columns when multiple aggs are used because
# it cant fit, for example, the sum, mean and median of proxy_mass_kg into a single column name
# pandas compensates by creating a multi index column that becomes annoying to access later
# ie: ('proxy_mass_kg', 'sum'), ('proxy_mass_kg', 'mean'), etc.
# access before flatten:
#   category_breakdown[('proxy_mass_kg', 'sum')]
#
# after flatten:
#   category_breakdown['Total Mass (kg)']
#
# we could work with it without flattening it, and in some siutations i actually like multi index columns,
# but for these reports its easier to read and work with if we flatten it.
category_breakdown.columns = category_breakdown.columns.droplevel(0) # drop the first level of the multi index
category_breakdown.columns = ['Count', 'Total Mass (kg)', 'Avg Mass (kg)', 'Median Mass (kg)', 
                              'Total Kinetic Energy (J)', 'Avg Kinetic Energy (J)', 'Avg Launch Year']

# in order for us to be able to calculate the percent of total of each category, we need the overall totals
# for these 3 metrics.
total_count = category_breakdown['Count'].sum()
total_mass = category_breakdown['Total Mass (kg)'].sum()
total_energy = category_breakdown['Total Kinetic Energy (J)'].sum()

# this calculates the total percentages for each category
# calculates as (category total / overall total) * 100 and then we round to 2 decimal places
# was '% of Count' etc but changed to 'Percent' to be consistent with other reports
category_breakdown['Percent Count'] = (category_breakdown['Count'] / total_count * 100).round(2)
category_breakdown['Percent Mass'] = (category_breakdown['Total Mass (kg)'] / total_mass * 100).round(2)
category_breakdown['Percent Energy'] = (category_breakdown['Total Kinetic Energy (J)'] / total_energy * 100).round(2)

print("\n" + "="*150)
print("CATEGORY BREAKDOWN (In-Orbit Objects Only)")
print("="*150)
print(category_breakdown.to_string())
print(f"\nTOTALS:")
print(f"  Objects: {int(total_count):,}")
print(f"  Mass: {total_mass:,.0f} kg ({total_mass/1e6:.2f} kilotons)")
print(f"  Kinetic Energy: {total_energy:.2e} J ({total_energy/1e12:.2f} TJ)")


CATEGORY BREAKDOWN (In-Orbit Objects Only)
                    Count  Total Mass (kg)  Avg Mass (kg)  Median Mass (kg)  Total Kinetic Energy (J)  Avg Kinetic Energy (J)  Avg Launch Year  Percent Count  Percent Mass  Percent Energy
category                                                                                                                                                                                   
Active Satellite    12469       6251893.00         501.39             355.0              1.398856e+14            1.121867e+10          2023.24          37.97         42.80           48.30
Debris              12654        668255.00          52.81              50.0              1.641591e+13            1.297290e+09          1993.72          38.53          4.57            5.67
Inactive Satellite   5261       2883144.85         548.02             355.0              4.756972e+13            9.041954e+09          2000.74          16.02         19.74           16.43
Rocket Body     

## 3. Operator Intelligence Report

In [4]:
# Top Operators Analysis
operator_intel = in_orbit.groupby('country_operator').agg({
    # heres the same thing as the category breakdown, but now grouped by country_operator
    # trying to think of a way to refactor this into a reusable function for all reports
    # which will help avoid code duplication and 'bloat' in the notebook
    'norad_id': 'count',
    'proxy_mass_kg': ['sum', 'mean'],
    'kinetic_joules': 'sum',
    'is_zombie': 'sum'
}).round(2)

operator_intel.columns = ['Object Count', 'Total Mass (kg)', 'Avg Mass (kg)',
                          'Total Kinetic Energy (J)', 'Zombie Count']

operator_intel = operator_intel.sort_values('Total Mass (kg)', ascending=False)

# Add percentages for operator intelligence report
operator_intel['Percent Objects'] = (operator_intel['Object Count'] / operator_intel['Object Count'].sum() * 100).round(2)
operator_intel['Percent Mass'] = (operator_intel['Total Mass (kg)'] / operator_intel['Total Mass (kg)'].sum() * 100).round(2)
operator_intel['Percent Zombie Rate'] = (operator_intel['Zombie Count'] / operator_intel['Object Count'] * 100).round(2)

print("\n" + "="*160)
print("OPERATOR INTELLIGENCE REPORT (Top 20 by Mass)")
print("="*160)
print(operator_intel.head(20).to_string()) 



OPERATOR INTELLIGENCE REPORT (Top 20 by Mass)
                      Object Count  Total Mass (kg)  Avg Mass (kg)  Total Kinetic Energy (J)  Zombie Count  Percent Objects  Percent Mass  Percent Zombie Rate
country_operator                                                                                                                                              
USA                           3650       2126724.75         582.66              3.859301e+13          1380            65.32         43.79                37.81
MULTINATIONAL                   62        692221.00       11164.85              1.451974e+13            31             1.11         14.25                50.00
CHINA                          465        493589.80        1061.48              5.749817e+12           293             8.32         10.16                63.01
RUSSIA                         163        266416.00        1634.45              3.649696e+12           110             2.92          5.49                67.48

## 4. Risk Profile Report

In [5]:
# Altitude Band Risk Analysis

# create an array of altitude bins and labels for categorization
# bins = 10 total indexes, 0-9
# labels = 9 labels, one for each bin range
# bins are like fense posts, labels are the gaps between the fense posts
altitude_bins = [0, 200, 400, 600, 800, 1000, 1500, 2000, 36000, 100000]
altitude_labels = ['<200km', '200-400km', '400-600km', '600-800km', '800-1000km',
                   '1000-1500km', '1500-2000km', '2000-36000km', '>36000km']

# a series is basically a single column dataframe
# so in this case we are creating a new column in the in_orbit dataframe, 'altitude_band'
# and we are assigning it the value of the series returned by pd.cut()
# pd.cut() takes a continuous variable (perigee_km) and bins it into discrete categories (altitude bands)
# pd.cut(df['column'], bins=bins, labels=labels)
in_orbit['altitude_band'] = pd.cut(in_orbit['perigee_km'], bins=altitude_bins, labels=altitude_labels)

# now that we have our altitude bands, we can group by them and calculate obj count, mass, and energy for each altitude band
# the altitude labels actually become the index of the resulting dataframe
altitude_risk = in_orbit.groupby('altitude_band', observed=False).agg({
    'norad_id': 'count',
    'proxy_mass_kg': 'sum',
    'kinetic_joules': 'sum',
    'category': lambda x: x.value_counts().to_dict()
}).round(2)

altitude_risk.columns = ['Object Count', 'Total Mass (kg)', 'Total Kinetic Energy (J)', 'Category Distribution']

print("\n" + "="*130)
print("ALTITUDE BAND RISK PROFILE")
print("="*130)
# foreach( altitude_band band in altitude_labels )
#   if band exists
#     select the row where index == band and assign it to the row variable
#       print details
for band in altitude_labels:
    if band in altitude_risk.index:
        row = altitude_risk.loc[band]
        print(f"\n{band}:")
        print(f"  Objects: {int(row['Object Count']):,}")
        print(f"  Mass: {row['Total Mass (kg)']:,.0f} kg")
        print(f"  Kinetic Energy: {row['Total Kinetic Energy (J)']:.2e} J")


ALTITUDE BAND RISK PROFILE

<200km:
  Objects: 75
  Mass: 112,513 kg
  Kinetic Energy: 1.48e+12 J

200-400km:
  Objects: 1,969
  Mass: 1,221,583 kg
  Kinetic Energy: 2.45e+13 J

400-600km:
  Objects: 13,412
  Mass: 5,272,152 kg
  Kinetic Energy: 1.49e+14 J

600-800km:
  Objects: 6,294
  Mass: 1,391,568 kg
  Kinetic Energy: 3.64e+13 J

800-1000km:
  Objects: 3,886
  Mass: 1,107,250 kg
  Kinetic Energy: 2.90e+13 J

1000-1500km:
  Objects: 3,592
  Mass: 972,789 kg
  Kinetic Energy: 2.33e+13 J

1500-2000km:
  Objects: 294
  Mass: 124,644 kg
  Kinetic Energy: 1.78e+12 J

2000-36000km:
  Objects: 2,782
  Mass: 3,886,002 kg
  Kinetic Energy: 2.18e+13 J

>36000km:
  Objects: 523
  Mass: 511,063 kg
  Kinetic Energy: 2.34e+12 J


## 5. Temporal Analysis

In [6]:
# Launch Year Trends
# launch_year becomes the rows, agg count sum sum
temporal_analysis = in_orbit.groupby('launch_year').agg({
    'norad_id': 'count',
    'proxy_mass_kg': 'sum',
    'kinetic_joules': 'sum'
}).round(2)

# rename columns for clarity, we only need these columns for the report so it doesn't matter
temporal_analysis.columns = ['Objects Launched', 'Total Mass (kg)', 'Total Kinetic Energy (J)']
temporal_analysis = temporal_analysis.sort_index(ascending=False)

# new line + (=*100) ====================================================================================================
print("\n" + "="*100)
print("TEMPORAL ANALYSIS: Top 20 Launch Years by Objects")
print("="*100)
print(temporal_analysis.sort_values('Objects Launched', ascending=False).head(20).to_string())

# calculate basic statistics on satellite age (in years)
# only consider non-null values for this 
age_mask = in_orbit['sat_age_years'].notna()

# use the mask we just made to filter the objects so we only describe non-null ages
age_stats = in_orbit[age_mask]['sat_age_years'].describe()

print("\n" + "="*100)
print("SATELLITE AGE STATISTICS (In-Orbit Objects)")
print("="*100)
print(age_stats.to_string())


TEMPORAL ANALYSIS: Top 20 Launch Years by Objects
             Objects Launched  Total Mass (kg)  Total Kinetic Energy (J)
launch_year                                                             
2025                     4555        1765090.0              4.928355e+13
2024                     3307        1087645.0              3.004898e+13
1999                     2648         250820.0              5.502471e+12
2022                     2582         757240.0              1.869560e+13
2023                     2470         953529.0              2.480634e+13
2021                     1168         473307.0              1.011987e+13
1993                      837         132975.0              2.561791e+12
1981                      647         140905.0              2.829711e+12
2020                      593         263469.0              4.965199e+12
2018                      534         349170.6              4.775424e+12
2000                      524         204441.0              2.700215e+12


## 6. Satellite Age & Zombie Profile

In [7]:
# Filter for zombies
zombies = in_orbit[in_orbit['is_zombie'] == 1].copy()

# Satellite Age Distribution
satellites = in_orbit[in_orbit['category'] == 'Active Satellite'].copy()
inactive_sats = in_orbit[in_orbit['category'] == 'Inactive Satellite'].copy()

print("\n" + "="*100)
print("SATELLITE POPULATION ANALYSIS")
print("="*100)
print(f"\nActive Satellites: {len(satellites):,}")
print(f"Inactive Satellites: {len(inactive_sats):,}")
print(f"Total Satellites: {len(satellites) + len(inactive_sats):,}")
print(f"\nZombie Satellites (Inactive + Age > 110% Design Life): {len(zombies):,}")
print(f"Zombie Rate: {len(zombies) / (len(satellites) + len(inactive_sats)) * 100:.2f}%")
print(f"\nZombie Age Statistics:")
print(zombies[['sat_age_years']].describe().to_string())

# Age distribution bins
# define the bins, then define the label that we will eventually bind to it (age_dist_named series)
age_bins = [0, 10, 15, 20, 25, 30, 40, 50, 100]
age_labels = ['0-10yr', '10-15yr', '15-20yr', '20-25yr', '25-30yr', '30-40yr', '40-50yr', '>50yr']

# value_counts with bins will automatically bin the values into the specified bins
age_dist = zombies['sat_age_years'].value_counts(bins=age_bins, sort=False).sort_index()
age_dist_named = pd.Series(age_dist.values, index=age_labels[:len(age_dist)])

print("\n" + "="*100)
print("ZOMBIE SATELLITE AGE DISTRIBUTION")
print("="*100)
print(age_dist_named.to_string())


SATELLITE POPULATION ANALYSIS

Active Satellites: 12,469
Inactive Satellites: 5,261
Total Satellites: 17,730

Zombie Satellites (Inactive + Age > 110% Design Life): 5,261
Zombie Rate: 29.67%

Zombie Age Statistics:
       sat_age_years
count    5261.000000
mean       25.256605
std        18.116436
min         1.000000
25%         6.000000
50%        24.000000
75%        40.000000
max        68.000000

ZOMBIE SATELLITE AGE DISTRIBUTION
0-10yr     1751
10-15yr     378
15-20yr     338
20-25yr     274
25-30yr     435
30-40yr     777
40-50yr     719
>50yr       589


## Summary Tables


In [8]:
# BASIC SUMMARY STATISTICS

# uses varibles declared earlier in the preceeding reports,
# specifically: master, in_orbit, satellites, inactive_sats, zombies
print("\n" + "="*80)
print("SUMMARY STATISTICS")
print("="*80)
print(f"Total records: {len(master):,}")
print(f"In-orbit objects: {len(in_orbit):,}")
# master - in_orbit = decayed
print(f"Decayed objects: {len(master) - len(in_orbit):,}") 

# formula: metric sum / 1e6 for kilotons, 1e12 for TJ
# in scientific notation, 1e6 = 1,000,000, 1e12 = 1,000,000,000,000
# using scientific notation makes it easier to read and write large numbers without counting zeros
print(f"Total in-orbit mass: {in_orbit['proxy_mass_kg'].sum() / 1e6:.2f} kilotons")
print(f"Total kinetic energy: {in_orbit['kinetic_joules'].sum() / 1e12:.2f} TJ")
print(f"Average velocity: {in_orbit['velocity_kms'].mean():.2f} km/s")

print("\nCategories:")
print(f"  Active Satellites: {len(in_orbit[in_orbit['category'] == 'Active Satellite']):,}")
print(f"  Inactive Satellites: {len(in_orbit[in_orbit['category'] == 'Inactive Satellite']):,}")
print(f"  Debris: {len(in_orbit[in_orbit['category'] == 'Debris']):,}")
print(f"  Rocket Bodies: {len(in_orbit[in_orbit['category'] == 'Rocket Body']):,}")

print("\nZombie satellites:")
print(f"  Total: {len(zombies):,}")
print(f"  Rate: {len(zombies) / (len(satellites) + len(inactive_sats)) * 100:.2f}%")
print(f"  Avg age: {zombies['sat_age_years'].mean():.1f} years")
print(f"  US: {len(zombies[zombies['country_operator'] == 'US']):,}")
print(f"  Russia/CIS: {len(zombies[zombies['country_operator'].isin(['CIS', 'Russia'])]):,}")

print("\nRisk hotspots:")
# im not using a varible to store this mask because its only used once here
# but if we wanted to we could do something like:
# OUTPUT_MASK = (in_orbit['perigee_km'] >= 400) & (in_orbit['perigee_km'] <= 600)
# print(f"  Objects in 400-600km: {len(in_orbit[output_mask]):,}")
print(f"  Objects in 400-600km: {len(in_orbit[(in_orbit['perigee_km'] >= 400) & (in_orbit['perigee_km'] <= 600)]):,}")
print(f"  Rocket bodies in 400-600km: {len(in_orbit[(in_orbit['category'] == 'Rocket Body') & (in_orbit['perigee_km'] >= 400) & (in_orbit['perigee_km'] <= 600)]):,}")
print(f"  Rocket bodies below 600km: {len(in_orbit[(in_orbit['category'] == 'Rocket Body') & (in_orbit['perigee_km'] < 600)]):,}")


SUMMARY STATISTICS
Total records: 32,838
In-orbit objects: 32,838
Decayed objects: 0
Total in-orbit mass: 14.61 kilotons
Total kinetic energy: 289.59 TJ
Average velocity: 6.91 km/s

Categories:
  Active Satellites: 12,469
  Inactive Satellites: 5,261
  Debris: 12,654
  Rocket Bodies: 2,402

Zombie satellites:
  Total: 5,261
  Rate: 29.67%
  Avg age: 25.3 years
  US: 0
  Russia/CIS: 0

Risk hotspots:
  Objects in 400-600km: 13,423
  Rocket bodies in 400-600km: 494
  Rocket bodies below 600km: 892


i really need a better way to do diagnostic reports, or maybe just cut down all of the reporting to just a single report with the most important things